In [ ]:
# Download github repository
!git clone https://github.com/uw-cmg/RPV_model_benchmarking

In [ ]:
# Package imports
import pandas as pd
from RPV_model_benchmarking.models import *
from RPV_model_benchmarking.data import *
from RPV_model_benchmarking.metrics import *
from RPV_model_benchmarking.plots import *

In [ ]:
# Make all the model predictions on the full dataset

df = DataLoader().load_rpv_data()

preds, df = EONY().predict(df)

preds, df = OWAY().predict(df)

preds, df = JOWAY().predict(df, atr2fte=1.38e20, nn_model='Jacobs25')

preds, df = E900().predict(df)

preds, df = GBR().predict(df) 

preds, df = GKRR().predict(df)

preds, df = EnsembleNN_Jacobs23().predict(df, return_ebars=False)

preds, df = EnsembleNN_Jacobs24().predict(df, return_ebars=False)

preds, df = EnsembleNN_Jacobs25().predict(df, return_ebars=False)


In [ ]:
# Just keep high fluence data and save the preds file

df = Metrics().filter_df(df, column_name='fluence_n_cm2', filter_val=6e19, filter_operation='greater')
df = Metrics().filter_df(df, column_name='datatype', filter_val='Plotter', filter_operation='equal')
df

In [ ]:
df.to_csv('RPV_preds_highfluence.csv', index=False)

In [ ]:
# Get model residuals and noise plot

print(df.shape)
df = Metrics().filter_df(df, column_name='Measured DT41J  [C]', filter_val=50, filter_operation='greater')
print(df.shape)

plot_residuals(trues=df['Measured DT41J  [C]'], 
               preds=df['Jacobs25 NN ensemble predicted TTS (degC)'], 
               name='Jacobs25')

plot_residuals(trues=df['Measured DT41J  [C]'], 
               preds=df['Jacobs23 NN ensemble predicted TTS (degC)'], 
               name='Jacobs23')

plot_model_noise(trues=df['Measured DT41J  [C]'], noise=9, num_points=df.shape[0])

In [ ]:
# Get the 5fold CV benchmarking results

data = Benchmarking().get_5fold_benchmarks(model_name='Jacobs25')
data

In [ ]:
# Make best and worst alloy ranking plot

df_best, df_best_sort = plot_best_worst_alloys(model_name='Jacobs25', num_alloys=25, metric='RMSE', plot_type='best')

df_worst, df_worst_sort = plot_best_worst_alloys(model_name='Jacobs25', num_alloys=25, metric='RMSE', plot_type='worst')

In [ ]:
# Make embrittlement curve

plot_embrittlement_curve(df, 
                         model_name_list=['Jacobs23', 'Jacobs24', 'Jacobs25'], 
                         flux_list=[3*10**10, 3.68*10**12], 
                         style='linear', 
                         num_points=50, 
                         ymax=350,
                        include_ATR2=True)

In [ ]:
# Make flux effect crossover histogram

df = DataLoader().load_rpv_data()

model_name_list = ['Jacobs23', 'Jacobs24', 'Jacobs25']
fluence = 6*10**19 # 6*10**19, 1*10**20
plot_flux_effect_crossover_histogram(df, model_name_list, fluence=fluence, flux_plotter=3*10**10, flux_atr2=3.68*10**12)

In [ ]:
# Make crossplots

df = DataLoader().load_rpv_data()

plot_crossplot(df, 
               #model_name_list=['GKRR'],
               #crossplot_variable_name = 'at_percent_Cu',
               crossplot_variable_name='wt_percent_Cu',
               model_name_list=['GBR', 'OWAY', 'Jacobs23'],
               crossplot_variable_min=0,
               crossplot_variable_max=0.8,
               grid_variables = ['wt_percent_Ni'],
               grid_values = [[0.4, 0.8, 1.2]],
               #grid_variables=['log(flux_n_cm2_sec)', 'wt_percent_Ni'],
               #grid_values=[[np.log10(3e10), np.log10(3.68e12)], [0.4, 0.8]],
               num_points=50,
               ymax=250)